In [58]:
import numpy as np
import pandas as pd
import unidecode as u
import geonamescache as g
import re

def cleanCity(c):
    uc = u.unidecode(c)
    firstBracket = uc.find('(')
    if firstBracket > 0:
        uc = uc[:firstBracket]
    return uc.strip()
    
gc = g.GeonamesCache()
countries = [u.unidecode(c['name']) for c in gc.get_countries().values()]
countries.sort(key=lambda c: -len(c))
cities = [cleanCity(c['name']) for c in gc.get_cities().values()]
cities.sort(key=lambda c: -len(c))

regexp1 = re.compile(r"\b(" + "|".join(countries) + r")\b")
regexp2 = re.compile(r"\b(" + "|".join(cities) + r")\b")

fdata = open('data/headlines.txt')
headlines = [u.unidecode(line[:-1]) for line in fdata]

def matchStr(m):
    if len(m) == 0:
        return np.NaN
    elif len(m) == 1:
        return m[0]
    elif m[0] == 'Can' or m[0] == 'Spring':
        return m[1]
    else:
        return m[0]

d_countries = [matchStr(re.findall(regexp1, h)) for h in headlines]
d_cities = [matchStr(re.findall(regexp2, h)) for h in headlines]



df = pd.DataFrame({'headline': headlines, 'city': d_cities, 'country': d_countries})
df.to_pickle('./data.pkl')